In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import scipy
import threading
import python_speech_features
from scipy.io import wavfile
import scipy.fftpack as fft
from scipy.signal import get_window
import IPython.display as ipd
import math
%matplotlib inline
import librosa
import pickle
from os import listdir
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import SimpleRNN
from keras.layers import Dropout
from os.path import isfile, join

In [ ]:
workDir = "/Volumes/ANKUR'S/Data for audio/voxcelebDataset"
trainDir = "free-spoken-digit-dataset-master/recordings/"
testDir = ""
trainFiles = [f for f in listdir(trainDir) if isfile(join(trainDir, f))]
# testFiles = [f for f in listdir(testDir) if isfile(join(testDir, f))]
print(len(trainFiles))
# print(len(testFiles))
# metaFile = "FSDKaggle2018.meta/train_post_competition.csv"
# meta = pd.read_csv(metaFile)

def returnMelCoeff(signal, sr):
    return librosa.feature.mfcc(signal, sr)

def windowHann(signal, fr, windowSize):
    window = get_window("hann", windowSize)
    windowed = signal * window
    return windowed

def padSignal(signal, sr, windowSize, hopSize):
    signalLength = len(signal)
    numZeros = windowSize - (signalLength - math.floor(signalLength/hopSize)*hopSize);
    return np.concatenate((signal, np.zeros(numZeros)), axis=None)

def process(signal, sr, windowSize, hopSize):
    origLength = len(signal)
    signal = padSignal(signal, sr, windowSize, hopSize)
    mfccArray = []
    for i in range(0, origLength, hopSize):
        toProcess = windowHann(signal[i: i+windowSize], sr, windowSize)
        mfccs = python_speech_features.mfcc(toProcess, sr, winlen=0.092, nfft = 2048, winstep=0.092)
        mfccArray.append(mfccs[0].tolist())
    return mfccArray

windowSize = 2048
hopSize = 256

with open('dumpfileMNIST.pickle', 'wb') as handle:
    for i in trainFiles:
        fName = os.path.join(trainDir, i)
        signal, sr = librosa.load(fName)
        mfccArray = process(signal, sr, windowSize, hopSize)
        tag = i
        a = {tag: mfccArray}
        pickle.dump(a, handle, protocol=pickle.HIGHEST_PROTOCOL)
        print("Done with: ", i)

In [ ]:
inputShape = (2561, 1)

regressor = Sequential()

regressor.add(SimpleRNN(units = 20, activation='tanh', return_sequences=True, input_shape= inputShape))
regressor.add(Dropout(0.2))

regressor.add(SimpleRNN(units = 20, activation='tanh', return_sequences=True))
regressor.add(Dropout(0.2))

regressor.add(SimpleRNN(units = 20, activation='tanh', return_sequences=True))
regressor.add(Dropout(0.2))

regressor.add(SimpleRNN(units = 20))
regressor.add(Dropout(0.2))

regressor.add(Dense(units = 1))

regressor.compile(optimizer='adam', loss='mean_squared_error')
regressor.summary()

# regressor.fit(X_train, y_train, epochs=100, batch_size=32)
# buildModel()

In [ ]:
pickeledData = []
objects = []
with (open("MNIST_MFCC_Normal_20.pickle", "rb")) as openfile:
    while True:
        try:
            objects.append(pickle.load(openfile))
        except EOFError:
            break

In [ ]:
windowSize = 512
hopSize = 128
trainDir = "free-spoken-digit-dataset-master/recordings/"
trainFiles = [f for f in listdir(trainDir) if isfile(join(trainDir, f))]

def windowHann(signal, fr, windowSize):
    window = get_window("hann", windowSize)
    windowed = signal * window
    return windowed

def padSignal(signal, sr, windowSize, hopSize):
    signalLength = len(signal)
    numZeros = windowSize - (signalLength - math.floor(signalLength/hopSize)*hopSize);
    return np.concatenate((signal, np.zeros(numZeros)), axis=None)

def processMFCCs(signal, sr, windowSize, hopSize):
    origLength = len(signal)
    signal = padSignal(signal, sr, windowSize, hopSize)
    mfccArray = np.empty((math.ceil(origLength/hopSize), 20))
    block = 0
    for i in range(0, origLength, hopSize):
        toProcess = windowHann(signal[i: i+windowSize], sr, windowSize)
        mfccArray[block, :] = python_speech_features.mfcc(toProcess, sr, winlen=(len(toProcess)/sr), nfft = len(toProcess), winstep=(len(toProcess)/sr), numcep=20)[0]
        block+=1
    return mfccArray

# testName = trainFiles[2]
# fName = os.path.join(trainDir, testName)
# signal, sr = librosa.load(fName)
# print(len(signal))
# mfccArray = processWaveletMFCCs(signal, sr, windowSize, hopSize, levels[0], wavelets[0])
# print(np.shape(mfccArray))


fileName = "MNIST_MFCC_Normal_20.pickle"
with open(fileName, 'wb') as handle:
    for i in trainFiles:
        fName = os.path.join(trainDir, i)
        signal, sr = librosa.load(fName)
        mfccArray = processMFCCs(signal, sr, windowSize, hopSize)
        a = {i: mfccArray}
        pickle.dump(a, handle, protocol=pickle.HIGHEST_PROTOCOL)
        print("Done with: ", i)

In [ ]:
for i in test:
    arr = test[i]

In [ ]:
objects[0]

In [ ]:
mfccs = test[[*test][0]]
print(mfccs)

In [ ]:
label = [*test][0]
a = label.split('_')
print(a[0], a[2].split('.')[0])

In [ ]:
[*i[[*i][0]]][0]

In [ ]:
mfcc[[*label][0]]

In [ ]:
from itertools import chain
list(chain.from_iterable(test[[*test][0]][[*label][0]]))

In [ ]:
trainX = []
trainY = []
testX = []
testY = []
for i in objects:
    label = [*i][0].split('_')[0]
    num = int([*i][0].split('_')[2].split('.')[0])
    if num<=4:
        testX.append(np.asarray(i[[*i][0]], dtype=np.float32))
        testY.append(label)
    else:
        trainX.append(np.asarray(i[[*i][0]], dtype=np.float32))
        trainY.append(label)
#     trainX.append(list(chain.from_iterable(i[[*i][0]][[*i[[*i][0]]][0]])))

In [ ]:
print(len(trainX), len(trainY), len(testX), len(testY))

In [ ]:
with open('dumpfileMNISTdataset.pickle', 'rb') as handle:
    pickle.load(handle, protocol=pickle.HIGHEST_PROTOCOL)
    pickle.dump({"trainY":trainY}, handle, protocol=pickle.HIGHEST_PROTOCOL)
    pickle.dump({"testX":testX}, handle, protocol=pickle.HIGHEST_PROTOCOL)
    pickle.dump({"testY":testY}, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
trainX = np.asarray(trainX, dtype=object)

In [ ]:
trainY = np.asarray(trainY)
testX = np.asarray(testX)
testY = np.asarray(testY)

In [ ]:
for i in range(len(trainX)):
    trainX[i] = np.pad(trainX[i], (0, 2561-len(trainX[i])), 'constant', constant_values=(0))

In [ ]:
mlen = 0
for i in range(len(trainX)):
    trainX[i]

In [ ]:
trainX[0]

In [ ]:
val = max(trainDataInput, key=len)

In [ ]:
np.shape(trainX[0])

In [ ]:
regressor.fit(trainX, trainY, epochs=10, batch_size=32)

In [ ]:
trainY = np.asarray(trainY, dtype=np.float32)

In [ ]:
trainY = []
for i in trainDataOutput:
    trainY.append(setDict[i])

In [ ]:
len(trainY)

In [ ]:
trainX = []
for i in trainDataInput:
    trainX.append(np.pad(np.asarray(i, dtype = np.float32), (0, 7815-len(i)), 'constant', constant_values=(0)))

In [ ]:
trainX = np.asarray(trainX)

In [ ]:
np.shape(trainX)

In [ ]:
trainY = np.asarray(trainY, dtype=np.int32)

In [ ]:
trainX

In [ ]:
t = np.reshape(trainX[0], (2561,1))

In [ ]:
for i in range(len(trainX)):
    trainX[i] = np.reshape(trainX[i], (2561,1))

In [ ]:
trainX = np.reshape(trainX, (2700,1))

In [ ]:
trainY = np.reshape(trainY, (2700,1))

In [ ]:
trainY